## RNN from scratch in PyTorch

![](https://miro.medium.com/max/840/1*o65pRKyHxhw7m8LgMbVERg.png)

First, let’s build the computation graph for a single-layer RNN. Again, we are not concerned with the math for now, I just want to show you the PyTorch operations needed to build your RNN models.

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np

In [4]:
class SingleRNN(nn.Module):
    def __init__(self, n_inputs, n_neurons):
        super(SingleRNN, self).__init__()

        self.Wx = torch.randn(n_inputs, n_neurons)  # 4 x 1
        self.Wy = torch.randn(n_neurons, n_neurons) # 1 x 1

        self.b = torch.zeros(1, n_neurons) # 1 x 4
    
    def forward(self, X0, X1):
        self.Y0 = torch.tanh(torch.mm(X0, self.Wx) + self.b) # 4 x 1

        self.Y1 = torch.tanh(torch.mm(self.Y0, self.Wy) + 
                             torch.mm(X1, self.Wx) + self.b) # 4 x 1
        
        return self.Y0, self.Y1

The forward function computes two outputs — one for each time step (two overall). Note that we are using tanh as the non-linearity (activation function) via torch.tanh(...).

This is how the data is fed into RNN:

![](https://miro.medium.com/max/764/1*xCj9h3f2kekfqN_dMCpcag.png)

In [5]:
N_INPUT = 4
N_NEURONS = 1


X0_batch = torch.tensor([[0,1,2,0], [3,4,5,0], 
                         [6,7,8,0], [9,0,1,0]],
                        dtype = torch.float) #t=0 => 4 X 4

X1_batch = torch.tensor([[9,8,7,0], [0,0,0,0], 
                         [6,5,4,0], [3,2,1,0]],
                        dtype = torch.float) #t=1 => 4 X 4
model = SingleRNN(N_INPUT, N_NEURONS)

Y0_val, Y1_val = model(X0_batch, X1_batch)

In [7]:
print(Y0_val)
print(Y1_val)

tensor([[-0.9905],
        [-0.9425],
        [-0.6910],
        [ 1.0000]])
tensor([[ 1.0000],
        [-0.0332],
        [ 0.9997],
        [ 0.9986]])


### Increasing neurons in RNN Layer

![](https://miro.medium.com/max/840/1*KLBXIeszx_cqkYs3-EXHwg.png)

In [8]:
class BasicRNN(nn.Module):
    def __init__(self, n_inputs, n_neurons):
        super(BasicRNN, self).__init__()

        self.Wx = torch.randn(n_inputs, n_neurons) # n_inputs x n_neurons
        self.Wy = torch.randn(n_neurons, n_neurons) # n_neurons x n_neurons

        self.b = torch.zeros(1, n_neurons) # 1 x neurons

    def forward(self, X0, X1):
        self.Y0 = torch.tanh(torch.mm(X0, self.Wx) + self.b) # batch_size x n_neurons
        self.Y1 = torch.tanh(torch.mm(self.Y0, self.Wy) + torch.mm(X1, self.Wx) + self.b) # batch_size x n_neurons

        return self.Y0, self.Y1    

In [9]:
N_INPUT = 3 # number of features in input
N_NEURONS = 5 # number of units in layer

X0_batch = torch.tensor([[0,1,2], [3,4,5], 
                         [6,7,8], [9,0,1]],
                        dtype = torch.float) #t=0 => 4 X 3

X1_batch = torch.tensor([[9,8,7], [0,0,0], 
                         [6,5,4], [3,2,1]],
                        dtype = torch.float) #t=1 => 4 X 3

model = BasicRNN(N_INPUT, N_NEURONS)

Y0_val, Y1_val = model(X0_batch, X1_batch)

In [10]:
print(Y0_val)
print(Y1_val)

tensor([[-0.9998, -0.8551, -0.9390,  0.7864,  0.9697],
        [-1.0000, -0.9927, -1.0000, -0.6045,  1.0000],
        [-1.0000, -0.9997, -1.0000, -0.9856,  1.0000],
        [-0.9996,  0.9989, -1.0000, -1.0000,  1.0000]])
tensor([[-1.0000, -0.9955, -1.0000, -0.9974,  1.0000],
        [-0.9169,  0.7480,  0.9892,  0.9962, -0.8102],
        [-1.0000, -0.5568, -1.0000, -0.8928,  1.0000],
        [-1.0000,  0.9568, -0.9894,  0.1957,  1.0000]])
